In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import sklearn.ensemble as skens
import sklearn.metrics as skmetric
import sklearn.naive_bayes as sknb
import sklearn.tree as sktree
import matplotlib.pyplot as plt
#%matplotlib inline
#import seaborn as sns
#sns.set(style='white', color_codes=True, font_scale=1.3)
import sklearn.externals.six as sksix
#import IPython.display as ipd
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import os
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
one = pd.read_csv('icd_one.csv')
two = pd.read_csv('icd_two.csv')
three = pd.read_csv('icd_three.csv')
full = pd.read_csv('icd_full.csv')

In [3]:
def system_data_stream(full_set_first, ppd, otb):
    base_startrow = 1
    base_endrow = otb
    test_startrow = otb + 1
    test_endrow = test_startrow + ppd
    day = 1

    # inital base model
    rf_model = skens.RandomForestClassifier(n_estimators=10,oob_score=True, criterion='entropy')

    calendar = []

    while day < 30:
        # TRAIN THE MODEL ON t
        base = full_set_first.ix[base_startrow:base_endrow,:].copy()
        rf_model.fit(base, base.FIRST_ICD9_DIAGNOSIS)

        # MAKE PREDICTIONS on first set of p
        new_guys = full_set_first.ix[test_startrow:test_endrow,:].copy()
        predicted_labels = rf_model.predict(new_guys.ix[:,1:])

        # PROVIDE STATISTICS ON THE PREDICTIONS
        #diag_pat_one_test['predicted_rf_tree'] = predicted_labels
        accuracy = accuracy_score(new_guys['FIRST_ICD9_DIAGNOSIS'], predicted_labels)

        # record to the calendar
        calendar.append((day, accuracy))

        # REPLACE (A PERCENTAGE) OF THE ORIGINAL AMOUNT WITH 'NEW' PATIENTS
        base_startrow = base_startrow + ppd
        print(base_startrow)
        base_endrow = base_endrow + ppd
        print(base_endrow)
        test_startrow = base_endrow + 1
        print(test_startrow)
        test_endrow = test_startrow + ppd
        print(test_endrow)
        day = day + 1
        print(day)

    return(calendar)

In [4]:
# CHOOSE AN AMOUNT (i.e. number of patients per day)
p = 100

# CHOOSE AN AMOUNT (i.e. number of original train base)
t = 70000

In [6]:
# scramble one
one = one.sample(frac=1)
one = one[['FIRST_ICD9_DIAGNOSIS', 'FIRST_ITEMID2', 'CATS', 'FIRST_NUMBER',
       'LAST_ITEMID2', 'LAST_CATS', 'ICUFIRSTCAREUNIT',
       'ICULASTCAREUNIT', 'ICUFIRSTWARDID', 'ICULASTWARDID', 'ICU_LOS',
       'LANGUAGE', 'RELIGION', 'MARITAL_STATUS',
       'ETHNICITY', 'DIAGNOSIS', 'GENDER', 'LENGTHOFSTAY_DAYS']]
one.head()

,FIRST_ICD9_DIAGNOSIS,FIRST_ITEMID2,CATS,FIRST_NUMBER,LAST_ITEMID2,LAST_CATS,ICUFIRSTCAREUNIT,ICULASTCAREUNIT,ICUFIRSTWARDID,ICULASTWARDID,ICU_LOS,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,LENGTHOFSTAY_DAYS
42861,99662,781,1,121.0,128,4,4,4,4,4,2.3396,1,2,1,1,22,1,4
47010,49121,781,1,39.0,31,9,4,4,4,4,7.1957,1,4,1,1,598,1,12
36154,40300,226381,4,0.0,226228,1,1,1,1,1,1.2737,2,5,1,1,66,2,3
76149,25040,-1,1,0.0,-1,1,1,1,7,7,1.2236,1,2,1,1,4033,1,8
79299,0383,220228,1,7.0,226228,1,3,3,9,9,0.9680,2,4,2,1,22,2,15


In [7]:
#print(system_data_stream(one, p, t))
rf_model = skens.RandomForestClassifier(n_estimators=10,oob_score=True, criterion='entropy')

calendar = []

base_startrow = 1
base_endrow = t
test_startrow = t + 1
test_endrow = test_startrow + p
day = 1

print(base_startrow)
print(base_endrow)
print(test_startrow)
print(test_endrow)
print(day)

1
70000
70001
70101
1


In [8]:
# TRAIN THE MODEL ON t
base = one.iloc[base_startrow:base_endrow,:].copy()
rf_model.fit(base.ix[:,1:], base.ix[:,0])

base.head()

C:\Users\julie\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\julie\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\julie\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


,FIRST_ICD9_DIAGNOSIS,FIRST_ITEMID2,CATS,FIRST_NUMBER,LAST_ITEMID2,LAST_CATS,ICUFIRSTCAREUNIT,ICULASTCAREUNIT,ICUFIRSTWARDID,ICULASTWARDID,ICU_LOS,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,GENDER,LENGTHOFSTAY_DAYS
47010,49121,781,1,39.0,31,9,4,4,4,4,7.1957,1,4,1,1,598,1,12
36154,40300,226381,4,0.0,226228,1,1,1,1,1,1.2737,2,5,1,1,66,2,3
76149,25040,-1,1,0.0,-1,1,1,1,7,7,1.2236,1,2,1,1,4033,1,8
79299,0383,220228,1,7.0,226228,1,3,3,9,9,0.9680,2,4,2,1,22,2,15
82446,82009,220046,1,120.0,226228,2,1,1,8,8,3.8986,2,2,2,1,10439,1,4


In [9]:
# MAKE PREDICTIONS on first set of p
new_guys = one.ix[test_startrow:test_endrow,:].copy()
predicted_labels = rf_model.predict(new_guys.ix[:,1:])

# PROVIDE STATISTICS ON THE PREDICTIONS
#diag_pat_one_test['predicted_rf_tree'] = predicted_labels
accuracy = accuracy_score(new_guys.ix[:,0], predicted_labels)

# record to the calendar
calendar.append((day, accuracy))

# REPLACE (A PERCENTAGE) OF THE ORIGINAL AMOUNT WITH 'NEW' PATIENTS
base_startrow = base_startrow + p
print(base_startrow)
base_endrow = base_endrow + p
print(base_endrow)
test_startrow = base_endrow + 1
print(test_startrow)
test_endrow = test_startrow + p
print(test_endrow)
day = day + 1
print(day)

print(calendar)
print(new_guys.head())

C:\Users\julie\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\julie\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


101
70100
70101
70201
2
[(1, 0.9945201421800948)]
      FIRST_ICD9_DIAGNOSIS  FIRST_ITEMID2  CATS  FIRST_NUMBER  LAST_ITEMID2  \
70001                85221         220048     3           0.0        226228   
25823                 5845             27     4           0.0            39   
5955                  5780             27     4           0.0            27   
20193                 5712             27     1           0.0           128   
51361                 2798            159     3           0.0           128   

       LAST_CATS  ICUFIRSTCAREUNIT  ICULASTCAREUNIT  ICUFIRSTWARDID  \
70001          2                 3                3               3   
25823          3                 4                4               4   
5955           9                 1                1               8   
20193          4                 1                1               5   
51361          4                 1                1               7   

       ICULASTWARDID  ICU_LOS  LANGUAGE  RELIGIO

C:\Users\julie\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [12]:
rf_model = skens.RandomForestClassifier(n_estimators=10,oob_score=True, criterion='entropy')

# TRAIN THE MODEL ON t
base = one.iloc[base_startrow:base_endrow,:].copy()
print(base.head())
rf_model.fit(base.ix[:,1:], base.ix[:,0])

      FIRST_ICD9_DIAGNOSIS  FIRST_ITEMID2  CATS  FIRST_NUMBER  LAST_ITEMID2  \
50395                 2848             27    11           0.0            27   
70394                99664         226381     2           0.0        226228   
85064                  430         226543    11           0.0        226228   
66591                27401         220046     1         120.0        226228   
91598                 9351         226381     3           0.0        226228   

       LAST_CATS  ICUFIRSTCAREUNIT  ICULASTCAREUNIT  ICUFIRSTWARDID  \
50395          2                 3                3               9   
70394          2                 2                2               6   
85064          2                 3                3               3   
66591          1                 4                4               4   
91598          1                 1                1               8   

       ICULASTWARDID  ICU_LOS  LANGUAGE  RELIGION  MARITAL_STATUS  ETHNICITY  \
50395             

C:\Users\julie\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


MemoryError: could not allocate 1229455360 bytes

In [ ]:
# MAKE PREDICTIONS on first set of p
#new_guys = one.ix[test_startrow:test_endrow,:].copy()
#predicted_labels = rf_model.predict(new_guys.ix[:,1:])

# PROVIDE STATISTICS ON THE PREDICTIONS
#diag_pat_one_test['predicted_rf_tree'] = predicted_labels
#accuracy = accuracy_score(new_guys.ix[:,0], predicted_labels)

# record to the calendar
#calendar.append((day, accuracy))

# REPLACE (A PERCENTAGE) OF THE ORIGINAL AMOUNT WITH 'NEW' PATIENTS
base_startrow = base_startrow + p
print(base_startrow)
base_endrow = base_endrow + p
print(base_endrow)
test_startrow = base_endrow + 1
print(test_startrow)
test_endrow = test_startrow + p
print(test_endrow)
day = day + 1
print(day)

print(calendar)